In this section, we define the dataset class that loads our spectrogram images, apply necessary image transformations, and build a CNN classifier using a ResNet18 backbone.

We use transfer learning by loading a pretrained ResNet18 model and replacing its final fully connected layer to classify between Parkinson’s Disease (PD) and Non-PD classes.

# Rename Img and move to class folder

In [ ]:
!rm -rf '/content/dataset/non pd'
!rm -rf '/content/dataset/pd'

In [ ]:
import os

def rename_spectrograms(output_folder):
    class_counters = {'NONPD': 1, 'PD': 1}

    for class_folder in ['NONPD', 'PD']:
        class_path = os.path.join(output_folder, class_folder)
        if not os.path.isdir(class_path):
            print(f"Skipping missing folder: {class_path}")
            continue

        # Explicit check, avoid substring errors
        if class_folder.upper() == 'NONPD':
            class_label = 'NONPD'
        elif class_folder.upper() == 'PD':
            class_label = 'PD'
        else:
            continue  # skip any unexpected folders

        for patient_folder in os.listdir(class_path):
            patient_path = os.path.join(class_path, patient_folder)
            if not os.path.isdir(patient_path):
                continue

            for record_folder in os.listdir(patient_path):
                record_path = os.path.join(patient_path, record_folder)
                if not os.path.isdir(record_path):
                    continue

                # Look for any PNG files (ignore their original names)
                for file in os.listdir(record_path):
                    file_path = os.path.join(record_path, file)
                    if os.path.isfile(file_path) and file.lower().endswith('.png'):
                        new_name = f"{class_label}_SPECTROGRAM_{class_counters[class_label]}.png"
                        new_path = os.path.join(record_path, new_name)

                        os.rename(file_path, new_path)
                        print(f"Renamed: {file_path} → {new_path}")

                        class_counters[class_label] += 1

rename_spectrograms("/content/dataset")

Renamed: /content/dataset/NONPD/patient 9 non pd/recording 1/spectrogram.png → /content/dataset/NONPD/patient 9 non pd/recording 1/NONPD_SPECTROGRAM_1.png
Renamed: /content/dataset/NONPD/patient 9 non pd/recording 3/spectrogram.png → /content/dataset/NONPD/patient 9 non pd/recording 3/NONPD_SPECTROGRAM_2.png
Renamed: /content/dataset/NONPD/patient 9 non pd/recording 2/spectrogram.png → /content/dataset/NONPD/patient 9 non pd/recording 2/NONPD_SPECTROGRAM_3.png
Renamed: /content/dataset/NONPD/patient 4 non pd/recording 1/spectrogram.png → /content/dataset/NONPD/patient 4 non pd/recording 1/NONPD_SPECTROGRAM_4.png
Renamed: /content/dataset/NONPD/patient 4 non pd/recording 3/spectrogram.png → /content/dataset/NONPD/patient 4 non pd/recording 3/NONPD_SPECTROGRAM_5.png
Renamed: /content/dataset/NONPD/patient 4 non pd/recording 2/spectrogram.png → /content/dataset/NONPD/patient 4 non pd/recording 2/NONPD_SPECTROGRAM_6.png
Renamed: /content/dataset/NONPD/patient 16 non pd/recording 1/spectrog

# Remove to class folder and clear unused cache

In [ ]:
!mv /content/dataset/NONPD/*/*/* /content/dataset/NONPD
!mv /content/dataset/PD/*/*/* /content/dataset/PD

In [ ]:
import os
import shutil

parent_folders = ["/content/dataset/NONPD", "/content/dataset/PD"]

for parent_folder in parent_folders:
    for item in os.listdir(parent_folder):
        item_path = os.path.join(parent_folder, item)
        if os.path.isdir(item_path):
            shutil.rmtree(item_path)
            print(f"Deleted folder: {item_path}")

Deleted folder: /content/dataset/NONPD/patient 9 non pd
Deleted folder: /content/dataset/NONPD/patient 4 non pd
Deleted folder: /content/dataset/NONPD/patient 16 non pd
Deleted folder: /content/dataset/NONPD/patient 19 non pd
Deleted folder: /content/dataset/NONPD/patient 2 non pd
Deleted folder: /content/dataset/NONPD/patient 10 non pd
Deleted folder: /content/dataset/NONPD/patient 18 non pd
Deleted folder: /content/dataset/NONPD/patient 7 non pd
Deleted folder: /content/dataset/NONPD/patient 15 non pd
Deleted folder: /content/dataset/NONPD/patient 3 non pd
Deleted folder: /content/dataset/NONPD/patient 12 non pd
Deleted folder: /content/dataset/NONPD/patient 11 non pd
Deleted folder: /content/dataset/NONPD/patient 13 non pd
Deleted folder: /content/dataset/NONPD/patient 5 non pd
Deleted folder: /content/dataset/NONPD/patient 17 non pd
Deleted folder: /content/dataset/NONPD/patient 6 non pd
Deleted folder: /content/dataset/NONPD/patient 1 non pd
Deleted folder: /content/dataset/NONPD/

#Create Dataframe

In [ ]:
import pandas as pd

# Get file paths using shell
paths = !find /content/dataset/*/* -type f

# Prepare list for dataframe
data = []

for p in paths:
    if 'NONPD' in p:
        label = 'NONPD'
    elif 'PD' in p:
        label = 'PD'
    else:
        print("ALERT")

    data.append({'path': p, 'label': label})

# Build dataframe
dataset = pd.DataFrame(data, columns=['path', 'label'])

In [ ]:
dataset

,path,label
0,/content/dataset/NONPD/NONPD_SPECTROGRAM_10.png,NONPD
1,/content/dataset/NONPD/NONPD_SPECTROGRAM_11.png,NONPD
2,/content/dataset/NONPD/NONPD_SPECTROGRAM_12.png,NONPD
3,/content/dataset/NONPD/NONPD_SPECTROGRAM_13.png,NONPD
4,/content/dataset/NONPD/NONPD_SPECTROGRAM_14.png,NONPD
...,...,...
86,/content/dataset/PD/PD_SPECTROGRAM_5.png,PD
87,/content/dataset/PD/PD_SPECTROGRAM_6.png,PD
88,/content/dataset/PD/PD_SPECTROGRAM_7.png,PD
89,/content/dataset/PD/PD_SPECTROGRAM_8.png,PD
